# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0


In [7]:
df.columns

Index(['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year'],
      dtype='object')

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()[0]
song_data

['SOUDSGM12AC9618304',
 'Insatiable (Instrumental Version)',
 'ARNTLGG11E2835DDB9',
 0,
 266.39628]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
df

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0


In [11]:
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0


In [12]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values.tolist()[0]
artist_data

['ARNTLGG11E2835DDB9', 'Clp', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [13]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [14]:
df

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0


In [15]:
log_files = get_files('data/log_data')

In [16]:
filepath = log_files[0]

In [17]:
df = pd.read_json(filepath, lines=True) 
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Mitch Ryder & The Detroit Wheels,Logged In,Tegan,F,65,Levine,205.03465,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Jenny Take A Ride (LP Version),200,1543363215796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
1,The Spill Canvas,Logged In,Tegan,F,66,Levine,358.03383,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,The TIde (LP Version),200,1543363420796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
2,Mogwai,Logged In,Tegan,F,67,Levine,571.19302,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Two Rights Make One Wrong,200,1543363778796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
3,Spor,Logged In,Tegan,F,68,Levine,380.34240,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Way Of The Samurai,200,1543364349796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
4,DJ Dizzy,Logged In,Tegan,F,69,Levine,221.15220,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Sexy Bitch,200,1543364729796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80


In [18]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [19]:
df = df[df['page'] == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Mitch Ryder & The Detroit Wheels,Logged In,Tegan,F,65,Levine,205.03465,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Jenny Take A Ride (LP Version),200,1543363215796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
1,The Spill Canvas,Logged In,Tegan,F,66,Levine,358.03383,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,The TIde (LP Version),200,1543363420796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
2,Mogwai,Logged In,Tegan,F,67,Levine,571.19302,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Two Rights Make One Wrong,200,1543363778796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
3,Spor,Logged In,Tegan,F,68,Levine,380.34240,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Way Of The Samurai,200,1543364349796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
4,DJ Dizzy,Logged In,Tegan,F,69,Levine,221.15220,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Sexy Bitch,200,1543364729796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80


In [20]:
t = pd.to_datetime(df['ts'], unit= 'ms')
t.head()

0   2018-11-28 00:00:15.796
1   2018-11-28 00:03:40.796
2   2018-11-28 00:09:38.796
3   2018-11-28 00:19:09.796
4   2018-11-28 00:25:29.796
Name: ts, dtype: datetime64[ns]

In [21]:
time_data = [t.values, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday]
column_labels = ['TimeStamp', 'Hour', 'Day', 'Week', 'Month', 'Year', 'Weekday']

In [22]:
time_df = pd.DataFrame({lab: data for lab,data in zip(column_labels, time_data)})
time_df.head()

,TimeStamp,Hour,Day,Week,Month,Year,Weekday
0,2018-11-28 00:00:15.796,0,28,48,11,2018,2
1,2018-11-28 00:03:40.796,0,28,48,11,2018,2
2,2018-11-28 00:09:38.796,0,28,48,11,2018,2
3,2018-11-28 00:19:09.796,0,28,48,11,2018,2
4,2018-11-28 00:25:29.796,0,28,48,11,2018,2


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [23]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [24]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [25]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [26]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [27]:
cur.execute('rollback')

In [28]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [29]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
        print(results)
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = [pd.to_datetime(row.ts, unit='ms'), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent]
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [30]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [70]:
df_log = pd.DataFrame({})
for log in log_files:
    df_log = df_log.append(pd.read_json(log, lines=True))


In [71]:
df_log.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8260 entries, 0 to 14
Data columns (total 18 columns):
artist           6975 non-null object
auth             8260 non-null object
firstName        7965 non-null object
gender           7965 non-null object
itemInSession    8260 non-null int64
lastName         7965 non-null object
length           6975 non-null float64
level            8260 non-null object
location         7965 non-null object
method           8260 non-null object
page             8260 non-null object
registration     7965 non-null float64
sessionId        8260 non-null int64
song             6975 non-null object
status           8260 non-null int64
ts               8260 non-null int64
userAgent        7965 non-null object
userId           8260 non-null object
dtypes: float64(2), int64(4), object(12)
memory usage: 1.2+ MB


In [72]:
df_song = pd.DataFrame({})
for song in song_files:
    df_song = df_song.append(pd.read_json(song, lines=True), ignore_index=True)

In [73]:
df_song.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 10 columns):
artist_id           80 non-null object
artist_latitude     33 non-null float64
artist_location     80 non-null object
artist_longitude    33 non-null float64
artist_name         80 non-null object
duration            80 non-null float64
num_songs           80 non-null int64
song_id             80 non-null object
title               80 non-null object
year                80 non-null int64
dtypes: float64(3), int64(2), object(5)
memory usage: 6.3+ KB


In [75]:
df_song.columns, df_log.columns


(Index(['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
        'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year'],
       dtype='object'),
 Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
        'length', 'level', 'location', 'method', 'page', 'registration',
        'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
       dtype='object'))

In [83]:
artists_names = list(set(df_song['artist_name']).intersection(set(df_log['artist'])))

In [87]:
df_song_list = df_song[df_song['artist_name'].isin(artists_names)]

In [88]:
df_log_list = df_log[df_log['artist'].isin(artists_names)]

In [96]:
df_song_list.sort_values(['artist_name', 'title'])[['artist_name', 'title']]

,artist_name,title
35,Blue Rodeo,Floating
30,Elena,Setanta matins
10,Gob,Face the Ashes
16,Gob,Face the Ashes
59,Gwen Stefani,Harajuku Girls
40,Jimmy Wakely,Broken-Down Merry-Go-Round
56,Line Renaud,Der Kleine Dompfaff
41,Lionel Richie,Tonight Will Be Alright
43,Lionel Richie,Tonight Will Be Alright
62,Lupe Fiasco,Streets On Fire (Explicit Album Version)


In [97]:
df_log_list.sort_values(['artist', 'song'])[['artist', 'song']]

,artist,song
51,Blue Rodeo,Hasn't Hit Me Yet
483,Elena,Setanta matins
218,Gob,Ming Tran
431,Gwen Stefani,Danger Zone
87,Jimmy Wakely,I Love You So Much It Hurts
92,Line Renaud,Le Soir
155,Lionel Richie,Don't Wanna Lose You
80,Lionel Richie,Hello
442,Lionel Richie,Lady
294,Lionel Richie,Still
